In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('Student Profiles Wrangled.csv')

In [2]:
def categorize_gpa(row):
    gpa = row['GPA']
    if gpa < 1:
        return 'Less Than 1'
    elif gpa < 2 and gpa >= 1:
        return '1 - 2'
    elif gpa < 3 and gpa >= 2:
        return '2 - 3'
    elif gpa < 3.5 and gpa >= 3:
        return '3 - 3.5'
    else:
        return 'More Than 3.5'

df['GPA Category'] = df.apply(categorize_gpa, axis=1)

In [13]:
df

,STUDENT ID,SALUTATION,GENDER,NATIONALITY,DOB,HIGHEST QUALIFICATION,NAME OF QUALIFICATION AND INSTITUTION,DATE ATTAINED HIGHEST QUALIFICATION,DESIGNATION,INTAKE NO,...,REGISTRATION FEE,PAYMENT MODE,COURSE FEE,GPA,CITIZENSHIP_STATUS,date_diff,COURSE,Age,GPA Category,Age Group
0,2020/1101-013/001,Ms,F,Singaporean,1978-04-03,Certificate,"Certificate in Office Skills, ITE",2016-11-06,"Snr Associate, Client Services",13th,...,107,NETS,1712,3.0,L,456,1101,46,3 - 3.5,Middle Aged
1,2020/1101-013/002,Ms,F,Singaporean,1966-03-23,Certificate,"WSQ Higher Certificate in Human Resources, WPL...",2018-02-06,Admin Officer,13th,...,107,NETS,1712,2.5,L,456,1101,58,2 - 3,Middle Aged
2,2020/1101-013/003,Ms,F,Singaporean,1988-04-12,Certificate,"Nitec in Service Skills (Office), ITE",2016-11-06,Admin Assistant,13th,...,107,Bank Transfer,1712,2.0,L,456,1101,36,2 - 3,Middle Aged
3,2020/1101-014/004,Ms,F,Singaporean,1991-11-27,Degree,Bachelor Science (Facilities & Events Manageme...,2017-06-10,HR Administrator,14th,...,107,NETS,1212,2.5,L,456,1101,32,2 - 3,Millenial
4,2020/1101-014/005,Ms,F,Singaporean,1985-04-23,Diploma,"Diploma in Procurement and Supply Management, ...",2016-08-10,Purchasing Executive,14th,...,107,NETS,1712,2.4,L,456,1101,39,2 - 3,Middle Aged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,2019/5113-007/005,Ms,F,Singaporean,1973-01-10,Degree,Bachelor of Commerce (Accounting and Finance)/...,2017-11-05,"Head, Registry & Corp Admin",7th,...,107,Bank Transfer,5803,2.9,L,183,5113,51,2 - 3,Middle Aged
240,2019/5113-007/006,Ms,F,Permanent Resident,1982-09-30,Degree,Bachelor of Economics (Accounting)/\r\nUnivers...,2016-05-05,Finance Officer,7th,...,107,Bank Transfer,5803,2.0,L,183,5113,41,2 - 3,Middle Aged
241,2019/5113-006/003,Ms,F,Singaporean,1990-06-03,Degree,Bachelor of Science in Hotel Administration (H...,2015-06-08,HR Assistant,6th,...,107,Bank Transfer,5803,2.8,L,183,5113,34,2 - 3,Millenial
242,2019/5113-005/001,Ms,F,Singaporean,1981-11-14,Degree,Bachelor of Arts with Second Class Honours (Lo...,2014-08-04,Household Goods Coordinator,5th,...,107,Bank Transfer,5803,1.9,L,180,5113,42,1 - 2,Middle Aged


# Plotly Sunburst

In [3]:
# Prepare the data for the sunburst plot
# Create simplified age groups
df['Age Group'] = pd.cut(df['Age'], bins=[18, 35, 60], labels=['Millenial', 'Middle Aged'])

# Drop rows with missing values in the relevant columns
df.dropna(subset=['COURSE', 'Age Group', 'GPA'], inplace=True)

# Filter data to include only the top 5 courses by student count
top_courses = df['COURSE'].value_counts().nlargest(5).index
df_filtered = df[df['COURSE'].isin(top_courses)]

# Add a Count column for aggregation
df_filtered['Count'] = 1

# Group by course and age group and calculate average GPA and count
df_grouped = df_filtered.groupby(['COURSE', 'Age Group']).agg(
    Count=('Count', 'sum'),
    Avg_GPA=('GPA', 'mean')
).reset_index()

# Calculate average GPA for each level
course_gpa = df_filtered.groupby('COURSE')['GPA'].mean().reset_index(name='GPA_course')

# Merge the average GPAs into the grouped dataframe
df_grouped = df_grouped.merge(course_gpa, on='COURSE')

# Create the sunburst plot
fig = px.sunburst(df_grouped, path=['COURSE', 'Age Group'], values='Count',
                  color='Avg_GPA', 
                  hover_data={
                      'Avg_GPA': ':.2f',
                      'GPA_course': ':.2f'
                  })

# Update hover template to show average GPAs for each level
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>' +
                  'Count: %{value}<br>' +
                  'Course GPA: %{customdata[1]:.2f}<br>' +
                  'Age Group GPA: %{customdata[0]:.2f}'
)

# Update layout for better aesthetics and readability
fig.update_layout(
    title={
        'text': 'Course and Age with Average GPAs',
        'y':0.975,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='black'),
    font=dict(family='Arial, sans-serif', size=16, color='black'),
    margin=dict(t=40, l=0, r=0, b=10),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Show the plot
fig.show()

In [4]:
app = Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# Create initial figures
fig_bar = px.bar(df, x="COURSE", y="GPA", color="GENDER", barmode="group")
fig_pie_1 = px.pie(df, values='GPA', names='COURSE', title='Course Distribution')
fig_pie_2 = px.pie(df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

# Update layout for all figures
fig_bar.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])
fig_pie_1.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])
fig_pie_2.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='DAVI CA2 DashBoard',
        style={
            'backgroundColor': '#444444',
            'textAlign': 'center',
            'color': colors['text'],
            'padding': '10px'
        }
    ),
    
    html.Div(style={
        'display': 'flex',
        'padding': '10px'
    }, children=[
        # Sidebar with radio buttons
        html.Div(style={
            'backgroundColor': '#444444',
            'padding': '10px',
            'borderRadius': '5px',
            'width': '20%',
            'marginRight': '10px',
            'textAlign': 'center'
        }, children=[
            html.H3(
                children='Filter by Nationality',
                style={'color': colors['text']}
            ),
            dcc.RadioItems(
                id='radio-buttons',
                options=[{'label': 'All Data', 'value': 'ALL'}] + 
                        [{'label': nat, 'value': nat} for nat in df['NATIONALITY'].unique()],
                value='ALL',
                style={'color': colors['text']}
            )
        ]),
        
        # Two horizontal cards
        html.Div(style={
            'display': 'flex',
            'flexDirection': 'column',
            'width': '75%'
        }, children=[
            # First horizontal card
            html.Div(style={
                'backgroundColor': '#444444',
                'padding': '10px',
                'borderRadius': '5px',
                'marginBottom': '10px',
                'textAlign': 'center'
            }, children=[
                dcc.Graph(
                    id='example-graph-1',
                    figure=fig_bar
                )
            ]),
            
            # Second horizontal card split into two pie charts
            html.Div(style={
                'backgroundColor': '#444444',
                'padding': '10px',
                'borderRadius': '5px',
                'textAlign': 'center',
                'display': 'flex',
                'justifyContent': 'space-around'
            }, children=[
                html.Div(style={'width': '45%'}, children=[
                    dcc.Graph(
                        id='example-pie-1',
                        figure=fig_pie_1
                    )
                ]),
                html.Div(style={'width': '45%'}, children=[
                    dcc.Graph(
                        id='example-pie-2',
                        figure=fig_pie_2
                    )
                ])
            ])
        ])
    ])
])

# Define callback to update graphs
@app.callback(
    Output('example-graph-1', 'figure'),
    Output('example-pie-1', 'figure'),
    Output('example-pie-2', 'figure'),
    Input('radio-buttons', 'value')
)
def update_graphs(selected_value):
    if selected_value != 'ALL':
        filtered_df = df[df['NATIONALITY'] == selected_value]
    else:
        filtered_df = df

    fig_bar = px.bar(filtered_df, x="COURSE", y="GPA", color="GENDER", barmode="group")
    fig_pie_1 = px.pie(filtered_df, values='GPA', names='COURSE', title='Course Distribution')
    fig_pie_2 = px.pie(filtered_df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

    fig_bar.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])
    fig_pie_1.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])
    fig_pie_2.update_layout(plot_bgcolor=colors['background'], paper_bgcolor=colors['background'], font_color=colors['text'])

    return fig_bar, fig_pie_1, fig_pie_2

if __name__ == '__main__':
    app.run(debug=True)

In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

# Assuming you have already imported your data as 'df'

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# Create initial figures
fig_bar = px.bar(df, x="COURSE", y="GPA", color="GENDER", barmode="group")
fig_pie_1 = px.pie(df, values='GPA', names='COURSE', title='Course Distribution')
fig_pie_2 = px.pie(df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

# Update layout for all figures
for fig in [fig_bar, fig_pie_1, fig_pie_2]:
    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text']
    )

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("DAVI CA2 DashBoard", className="text-center mb-4"), width=12)
    ]),
    
    dbc.Row([
        # Sidebar with radio buttons
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H3("Nationality", className="text-center")),
                dbc.CardBody([
                    html.H6("Filters"),
                    dcc.RadioItems(
                        id='radio-buttons',
                        options=[{'label': 'All Data', 'value': 'ALL'}] + 
                                [{'label': nat, 'value': nat} for nat in df['NATIONALITY'].unique()],
                        value='ALL',
                        className="mb-2"
                    )
                ])
            ])
        ], width=3),
        
        # Main content area
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='example-graph-1', figure=fig_bar)
                ])
            ], className="mb-4"),
            
            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-1', figure=fig_pie_1)
                        ])
                    ])
                ], width=6),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-2', figure=fig_pie_2)
                        ])
                    ])
                ], width=6)
            ])
        ], width=9)
    ])
], fluid=True, style={'backgroundColor': colors['background']})

# Define callback to update graphs
@app.callback(
    Output('example-graph-1', 'figure'),
    Output('example-pie-1', 'figure'),
    Output('example-pie-2', 'figure'),
    Input('radio-buttons', 'value')
)
def update_graphs(selected_value):
    if selected_value != 'ALL':
        filtered_df = df[df['NATIONALITY'] == selected_value]
    else:
        filtered_df = df

    fig_bar = px.bar(filtered_df, x="COURSE", y="GPA", color="GENDER", barmode="group")
    fig_pie_1 = px.pie(filtered_df, values='GPA', names='COURSE', title='Course Distribution')
    fig_pie_2 = px.pie(filtered_df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

    for fig in [fig_bar, fig_pie_1, fig_pie_2]:
        fig.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
        )

    return fig_bar, fig_pie_1, fig_pie_2

if __name__ == '__main__':
    app.run_server(debug=True)

In [6]:
#Dash-barchart and dropdown menu
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
 
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# default theme is BOOTSTRAP. Refer to https://dash-bootstrap-components.opensource.faculty.ai/docs/themes/
 
poverty_data = pd.read_csv('PovStatsData.csv')
poverty = pd.read_csv('poverty.csv', low_memory=False)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

 
app.layout = html.Div([
    html.H1(children='Poverty And Equity Database', style={'textAlign': 'center'}),
    html.H2('The World Bank'),
    dcc.Dropdown(id='country',
                 options=[{'label': country, 'value': country}
                          for country in poverty_data['Country Name'].unique()]),
    html.Br(),
    html.Div(id='report'),
    html.Br(),
    dcc.Dropdown(id='year_dropdown',
                 value='2010',
                 options=[{'label': year, 'value': str(year)}
                          for year in df['COURSE'].unique()]),
    dcc.Graph(id='population_chart'),
 
    html.Br(),
])
 
 
@app.callback(Output('report', 'children'),
              Input('country', 'value'))
def display_country_report(country):
    if country is None:
        return ''
 
    filtered_df = poverty_data[(poverty_data['Country Name']==country) &
                               (poverty_data['Indicator Name']=='Population, total')]
    population = filtered_df.loc[:, '2010'].values[0]
 
    return [html.H3(country),
            f'The population of {country} in 2010 was {population:,.0f}.']
 
 
@app.callback(Output('population_chart', 'figure'),
              Input('year_dropdown', 'value'))

def plot_countries_by_population(year):
    fig = go.Figure()
    year_df = df[['COURSE', year]].sort_values(year, ascending=False)[:20]
    fig.add_bar(x=year_df['Country Name'],
                y=year_df[year])
    fig.layout.title = f'Top twenty countries by population - {year}'
    return fig
 
 
if __name__ == '__main__':
    app.run(debug=True, port=8053) #  app.run_server(debug=True, port=8051) # or whatever you choose

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[6], line 61, in plot_countries_by_population(year=None)
     56 @app.callback(Output('population_chart', 'figure'),
     57               Input('year_dropdown', 'value'))
     58 
     59 def plot_countries_by_population(year):
     60     fig = go.Figure()
---> 61     year_df = df[['COURSE', year]].sort_values(year, ascending=False)[:20]
        year = None
        df =             STUDENT ID SALUTATION GENDER         NATIONALITY         DOB  \
0    2020/1101-013/001         Ms      F         Singaporean  1978-04-03   
1    2020/1101-013/002         Ms      F         Singaporean  1966-03-23   
2    2020/1101-013/003         Ms      F         Singaporean  1988-04-12   
3    2020/1101-014/004         Ms      F         Singaporean  1991-11-27   
4    2020/1101-014/005         Ms      F         Singaporean  1985-04-23   
..        